Import required dependencies

In [ ]:
import os
import json
import requests
import pandas as pd
import pickle           # Save and load data
import time


# Setting Pandas options.
pd.set_option("display.max_rows", 50) # How to display all rows from data frame using pandas. Setting value to None to show all rows.
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 100)
pd.set_option("display.max_info_rows", 1000000)
pd.set_option("display.precision", 2)
#pd.set_option("styler.format.precision", 2)

load config files

In [ ]:
# Load API Key and download directory from config file
CONFIG_FILE = "../config/api-call.json"

def load_config(config_path):
    """Load configuration from a JSON file."""
    with open(config_path, "r") as file:
        return json.load(file)

Config parameters for API calls

In [ ]:
# Load config values
config = load_config(CONFIG_FILE)
# print(config)
API_ENDPOINT= config["ned"]["ned_api_endpoint"]
API_KEY = config["ned"]["demo-ned-api-key"]
DOWNLOAD_DIR = config["ned"]["ned_download_dir"]

# https://api.ned.nl/v1/utilizations?point=0&type=2&granularity=3&granularitytimezone=1&classification=2&activity=1&validfrom[strictly_before]=2020-11-17&validfrom[after]=2020-11-16


# Ensure the download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# API details (Replace with actual API endpoint)
API_URL = API_ENDPOINT
#HEADERS = {"Authorization": f"X-AUTH-TOKEN {API_KEY}"}

# Headers with X-AUTH-TOKEN
api_headers = {
    "X-AUTH-TOKEN": API_KEY,  # Replace with your actual token
    "Content-Type": "application/json"  # Optional, if sending JSON
}

Make API calls in loop

In [45]:

# Type	    What is the type of energy carrier?	0 All, 1 Wind, 2 Solar, 3 Biogas, 4 HeatPump, 8 Cofiring, 9 Geothermal, 10 Other, 11 Waste, 12 BioOil, 13 Biomass
# 14 Wood, 17 WindOffshore, 18 FossilGasPower, 19 FossilHardCoal, 20 Nuclear, 21 WastePower, 22 WindOffshoreB, 23 NaturalGas, 24 Biomethane, 25 BiomassPower
# 26 OtherPower, 27 ElectricityMix, 28 GasMix, 31 GasDistribution, 35 CHP Total, 50 SolarThermal, 51 WindOffshoreC, 53 IndustrialConsumersGasCombination
# 54 IndustrialConsumersPowerGasCombination, 55 LocalDistributionCompaniesCombination, 56 AllConsumingGas


# Define an array of n values (Custom values instead of a fixed range)
# n_values = [1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 35, 50, 51, 53, 54, 55, 56]  # Type values

n_values = [1, 2]  # Type values

# n_values = [0]  # Type values


# Loop through the n values
for gen_type in n_values:
    #params = {"n": n}  # Pass n as a parameter
    api_params={
        "point": "11",
        "type": gen_type,
        "granularity": "4",
        "granularitytimezone": "1",
        "classification": "2",
        "activity": "1",
        "validfrom[strictly_before]": "2025-02-28",
        "validfrom[after]": "2022-01-01"
        }

    """Download a data file from an API and save it in the specified directory."""
    response = requests.get(API_URL, headers=api_headers, params=api_params)

    print("response", response.status_code)

    # Convert JSON to Pandas DataFrame
        
    if response.status_code == 200 and response.text.strip():
        # JSON in bytes format
        json_bytes = response.content

        # Convert bytes → string → dictionary
        json_dict = json.loads(json_bytes.decode("utf-8"))

        print(json_dict)

        # Convert to DataFrame
        df = pd.DataFrame.from_dict(json_dict, orient="index")

        # Convert the list of utilizations into a DataFrame
        json_data = pd.DataFrame(json_dict["hydra:member"])

        # Display the DataFrame as a table
        print("\nConverted JSON Payload to Table Format:\n")
        print(json_data.to_string(index=False))

            
        file_path = os.path.join(DOWNLOAD_DIR, f"power-gen-type-{gen_type}.json")  # Change extension as needed
        with open(file_path, "wb") as file:
            file.write(response.content)
        print(f"File downloaded successfully: {file_path}")        
    else:
        print(f"Error for {gen_type} : {response.status_code} - {response.text}")
    #filtered_df = filter_response_data(response_data)
    time.sleep(1)  # Optional delay to avoid rate limits

response 200
{'@context': '/v1/contexts/Utilization', '@id': '/v1/utilizations', '@type': 'hydra:Collection', 'hydra:totalItems': 110784, 'hydra:member': [{'@id': '/v1/utilizations/23791261756', '@type': 'Utilization', 'id': 23791261756, 'point': '/v1/points/11', 'type': '/v1/types/1', 'granularity': '/v1/granularities/4', 'granularitytimezone': '/v1/granularity_time_zones/0', 'activity': '/v1/activities/1', 'classification': '/v1/classifications/2', 'capacity': 51208, 'volume': 12802, 'percentage': 0.1896238625049591, 'emission': 0, 'emissionfactor': 0, 'validfrom': '2021-12-31T23:00:00+00:00', 'validto': '2021-12-31T23:15:00+00:00', 'lastupdate': '2023-05-25T10:42:14+00:00'}, {'@id': '/v1/utilizations/23791591487', '@type': 'Utilization', 'id': 23791591487, 'point': '/v1/points/11', 'type': '/v1/types/1', 'granularity': '/v1/granularities/4', 'granularitytimezone': '/v1/granularity_time_zones/0', 'activity': '/v1/activities/1', 'classification': '/v1/classifications/2', 'capacity': 4

In [ ]:
# def filter_response_data(json_dict):

    # Convert the list of utilizations into a DataFrame
    json_data = pd.DataFrame(json_dict["hydra:member"])
    
    # f_describe(df, 10)
    #df_orig_num    = json_data.select_dtypes(include='number')
    #l_df_num_names = df_orig_num.columns.tolist()

    # print(l_df_num_names)
    # print(f"\nNumber of numerical variables: {len(l_df_num_names)}")

    #df_orig_cat    = json_data.select_dtypes(include='object')
    #l_df_cat_names = list(df_orig_cat.columns)

    # print(f"\nNumber of categorical variables: {len(l_df_cat_names)}")
    # print(l_df_cat_names)

    formatted_df = json_to_table(json_data)

    # Convert 'capacity' to numeric, handling errors by setting non-numeric values to NaN
    formatted_df['capacity'] = pd.to_numeric(formatted_df['capacity'], errors='coerce')

    # Filter, excluding rows where capacity is NaN
    filtered_df = formatted_df[formatted_df['capacity'] > 0].dropna(subset=['capacity'])

    print(filtered_df.shape[0])
    print(filtered_df)

    # Create dictionary 'dc_ned_json_data_1' with objects that will be used in the next exercises.
    #dc_ned_json_data_1 = {
    #    'df_orig': json_data    
    #}

    # Save dc_exercise_1_2_3 as 'dc_ned_json_data_1.pkl'
    #with open('../data/dc-ned-json-data-1.pkl', 'wb') as pickle_file:
    #    pickle.dump(dc_ned_json_data_1, pickle_file)

    # Display the DataFrame as a table
    # print("\nConverted JSON Payload to Table Format:\n")
    # print(json_data.to_string(index=False))

    #if formatted_df is not None:
    #    # Print the DataFrame (table format)
    #    print(formatted_df.head(3))
    return filtered_df

Functionl to make API calls

In [ ]:
# def json_to_table(json_data):
    #Converts JSON utilization data to a Pandas DataFrame for table display.
    # Args: json_data: The JSON data (either a string or a Python dictionary).
    # Returns: A Pandas DataFrame or None if there's an error.
    
    try:
        # If the input is a JSON string, parse it
        if isinstance(json_data, str):
            data = json.loads(json_data)
        elif isinstance(json_data, dict): # if it is already a dictionary
            data = json_data
        else:
            print("Invalid input: Please provide a JSON string or dictionary.")
            return None

        # Extract the 'hydra:member' array which contains the utilization data
        utilization_data = data.get('hydra:member', [])  # Handle missing key

        # Create a list of dictionaries, where each dictionary represents a row
        rows = []
        for item in utilization_data:
            # Select the fields you want to include in the table
            row = {
                'id': item.get('id'),
                'power-gen-type': item.get('type').split("/")[-1],
                #'capacity': item.get('capacity'),
                #'volume': item.get('volume'),
                #'percentage': item.get('percentage'),
                # 'emission': item.get('emission'),
                #'emissionfactor': item.get('emissionfactor'),
                # '@id': item.get('@id'),
                # '@type': item.get('@type'),
                # 'point': item.get('point'), # Include point for more context
                # 'type': item.get('type'), # Include type for more context
                #'granularity': item.get('granularity'), # Include granularity for more context
                #'granularitytimezone': item.get('granularitytimezone'), # Include granularity for more context
                # 'activity': item.get('activity'), # Include granularity for more context
                #'classification': item.get('classification'),
                # 'validfrom': item.get('validfrom'),
                'validto': item.get('validto')#,
                #'lastupdate': item.get('lastupdate')
            }
            rows.append(row)

        # Create a Pandas DataFrame from the list of dictionaries
        df = pd.DataFrame(rows)
        
        print("Number of rows : ", df.shape[0])
        print(df.head(3))

        return df

    except json.JSONDecodeError:
        print("Error: Invalid JSON format.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None